<a href="https://colab.research.google.com/github/JTT94/nlp_neural_process/blob/master/Class_BERT_NeuralProcesses_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re

from keras import backend as K
import numpy as np
import string
from datetime import datetime 
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

W0525 10:56:04.882961 140665273620352 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
!pip install bert-tensorflow
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization


     |████████████████████████████████| 71kB 3.4MB/s 


In [0]:
# initialiase tensorboard 
# from https://stackoverflow.com/questions/47818822/can-i-use-tensorboard-with-google-colab


# Get TensorBoard running in the background. 
LOG_DIR = './test_output'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [4]:
# # #Download and unzip ngrok. 
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


--2019-05-25 10:56:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.237.103, 52.203.53.176, 35.173.6.94, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.237.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  40.5MB/s    in 0.4s    

2019-05-25 10:56:13 (40.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
# #Launch ngrok background process...
get_ipython().system_raw('./ngrok http 6006 &')

In [6]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://fc7904a4.ngrok.io


In [7]:
import os, sys
sys.path.append('../') # add personal code dir to path for import


!test -d neural_process || git clone https://github.com/JTT94/nlp_neural_process.git neural_process
sys.path.append('./neural_process/')

import random
from neural_process import split_context_target, NeuralProcessParams
from neural_process.network import *
from neural_process.loss import *
from neural_process.predict import *
from neural_process.process import *

from neural_process.tf_model_builder_AUC import *
from neural_process.bert_utils import *

Cloning into 'neural_process'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 101 (delta 61), reused 97 (delta 59), pack-reused 0
Receiving objects: 100% (101/101), 66.24 KiB | 3.01 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [8]:
# Initialize session
sess = tf.Session()
K.set_session(sess)
K.tensorflow_backend._get_available_gpus()


['/job:localhost/replica:0/task:0/device:GPU:0']

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## For kaggle dataset
filename = './gdrive/My Drive/Data/train.csv'
df = pd.read_csv(filename)
cols = ['comment_text','toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df[cols]

score_column = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'] 
text_col_name = 'comment_text'

#Cast to float - because scores and labels need to be concattenated in the model function, and so need to be same type
for i in score_column:
  df[i] = pd.to_numeric(df[i],downcast='float')

In [11]:
# Restrict comment length
df = df[df.comment_text.str.len() <= 250]
df['num_toxic_atts'] = df[cols[1:]].apply(lambda x: np.sum(x), axis = 1)

df_toxic = df[df.num_toxic_atts  > 0]
# print(len(df_toxic))
df_healthy_sample = df[df.num_toxic_atts == 0][:len(df_toxic)]
print(len(df_healthy_sample))
df_healthy_remaining = df[df.num_toxic_atts == 0][len(df_toxic):]
print(len(df_healthy_remaining))

df_overrep = pd.concat([df_toxic, df_healthy_sample]).sample(frac=1.0)
print(len(df_overrep))
train_propn = 0.8

ratio = len(df_toxic)/(len(df_healthy_sample)+len(df_healthy_remaining))
print(ratio)

df_train = df_overrep[:int(len(df_overrep)*train_propn)]
print(len(df_train))

#construct a test set with similar proportion/imbalance in toxic - nontoxic data to the real data set
df_test = pd.concat([df_overrep[int(len(df_overrep)*train_propn):], df_healthy_remaining[:int((1-0.8)*(len(df_healthy_remaining)-len(df_train)))]])

print(0.5*len(df_overrep[int(len(df_overrep)*train_propn):]) / len(df_test))
# df_train.head(50)

11640
67417
23280
0.14723553891496008
18624
0.16150964340224783


In [12]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
tokenizer = create_tokenizer_from_hub_module(BERT_model_hub)

Instructions for updating:
Colocations handled automatically by placer.


W0525 10:57:06.193778 140665273620352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0525 10:57:08.206205 140665273620352 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [0]:
# #Pre process data for bert embedding

max_seq_length = 128

import pickle

train_input_examples = create_examples(df_train, score_column, text_col_name)
test_input_examples = create_examples(df_test, score_column, text_col_name)

train_features = convert_examples_to_features(train_input_examples, max_seq_length, tokenizer)
test_features = convert_examples_to_features(test_input_examples, max_seq_length, tokenizer)

pickle.dump(train_features, open('./gdrive/My Drive/Data/train_features.p', 'wb'))
pickle.dump(test_features, open('./gdrive/My Drive/Data/test_features.p', 'wb'))

# # ## Load features previously saved
# train_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))
# test_features = pickle.load(open('./gdrive/My Drive/Kaggle_toxic_comments/train_features.p', 'rb'))

In [0]:
# utility methods
from collections import namedtuple
NeuralProcessParams = namedtuple('NeuralProcessParams', ['dim_z', 'n_hidden_units_h', 'n_hidden_units_g'])
GaussianParams = namedtuple('GaussianParams', ['mu', 'sigma'])


def batch_mlp(input, inner_layer_dims, output_dim, variable_scope):
  """Apply MLP to the final axis of a 3D tensor (reusing already defined MLPs).
  
  Args:
    input: input tensor of shape [B,n,d_in].
    output_sizes: An iterable containing the output sizes of the MLP as defined 
        in `basic.Linear`.
    variable_scope: String giving the name of the variable scope. If this is set
        to be the same as a previously defined MLP, then the weights are reused.
    
  Returns:
    tensor of shape [B,n,d_out] where d_out=output_sizes[-1]
  """
  # Get the shapes of the input and reshape to parallelise across observations

  output = input

  
  # Pass through MLP
  with tf.variable_scope(variable_scope, reuse=tf.AUTO_REUSE):
    for i, size in enumerate(inner_layer_dims):
      output = tf.nn.relu(
          tf.layers.dense(output, size, name="layer_{}".format(i)))

    # Last layer without a ReLu
    output = tf.layers.dense(output, output_dim, name="layer_{}".format(i + 1))

  return output

In [0]:
class Embedder(object):
  
  def __init__(self, BERT_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", trainable=True):
    self.embedder = hub.Module(BERT_model_hub,trainable=trainable)
  
  def __call__(self, input_ids, input_mask, segment_ids):
    embedder = hub.Module(BERT_model_hub,trainable=trainable)
    bert_inputs = dict(input_ids=input_ids,
                       input_mask=input_mask, 
                       segment_ids=segment_ids)

    bert_outputs = self.embedder(inputs=bert_inputs,
                               signature="tokens", 
                               as_dict=True)
    
  # Use "pooled_output" for classification tasks on an entire sentence. Use "sequence_outputs" for token-level output
    return bert_outputs["pooled_output"]
  

In [0]:
class Decoder(object):
  """The Decoder."""

  def __init__(self, layer_dims, num_classes):
    self.layer_dims = layer_dims
    self.num_classes = num_classes
    
  def __call__(self, input_xs_embedding, z_samples):
  
        # inputs dimensions
    # z_sample has dim [n_draws, dim_z]
    # x_star has dim [N_star, dim_x]
    n_draws = z_samples.get_shape().as_list()[0]
    n_xs = tf.shape(input_xs_embedding)[0]

    # Repeat z samples for each x*
    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)

    #z_samples_repeat = tf.expand_dims(z_samples, axis=1)
    z_samples_repeat = tf.tile(z_samples, [1, n_xs, 1])

    # Repeat x* for each z sample
    x_star_repeat = tf.expand_dims(input_xs_embedding, axis=0)
    x_star_repeat = tf.tile(x_star_repeat, [n_draws, 1, 1])

    # Concatenate x* and z
    inputs = tf.concat([x_star_repeat, z_samples_repeat], axis=2)

    # decoder mlp
    inner_layer_dims = self.layer_dims
    output_dim = self.num_classes *2
    hidden = batch_mlp(inputs, inner_layer_dims, output_dim, "decoder")

    # Get the mean an the variance
    mu, log_sigma = tf.split(hidden, 2, axis= -1)

    # Bound the variance
    sigma_star = 0.1 + 0.9 * tf.nn.softplus(log_sigma)
    mu_star = tf.math.sigmoid(mu)


    return GaussianParams(mu_star, sigma_star)




In [0]:
class Encoder(object):

  def __init__(self, layer_dims, latent_dim):
    self.layer_dims = layer_dims
    self.latent_dim = latent_dim
    
  def __call__(self, xs, ys):
    
    xys = tf.concat([xs, ys], axis=1)


    # encoder mlp
    inner_layer_dims = self.layer_dims[:-1]
    output_dim = self.layer_dims[-1]
    rs = batch_mlp(xys, inner_layer_dims, output_dim, "encoder")
    
    # aggregate rs
    r = self._aggregate_r(rs)
    
    # get mu and sigma
    z_params = self._get_z_params(r)
    
    # distribution
    dist = tfd.MultivariateNormalDiag(loc=z_params.mu,
                                          scale_diag=z_params.sigma)
    return dist
    
  def _aggregate_r(self, context_rs: tf.Tensor) -> tf.Tensor:
    """Aggregate the output of the encoder to a single representation

    Creates an aggregation (mean) operator to combine the encodings of multiple context inputs

    Parameters
    ----------
    context_rs
        Input encodings tensor, shape: (n_samples, dim_r)

    Returns
    -------
        Output tensor of aggregation result
    """
    mean = tf.reduce_mean(context_rs, axis=0)
    r = tf.reshape(mean, [1, -1])
    return r
  
  def _get_z_params(self, context_r: tf.Tensor) -> GaussianParams:
    """Map encoding to mean and covariance of the random variable Z

    Creates a linear dense layer to map encoding to mu_z, and another linear mapping + a softplus activation for Sigma_z

    Parameters
    ----------
    context_r
        Input encoding tensor, shape: (1, dim_r)
    params
        Neural process parameters

    Returns
    -------
        Output tensors of the mappings for mu_z and Sigma_z
    """
    hidden = context_r
    with tf.variable_scope("latent_encoder", reuse=tf.AUTO_REUSE):
      # First apply intermediate relu layer 
      hidden = tf.nn.relu(
          tf.layers.dense(hidden, 
                          (self.layer_dims[-1] + self.latent_dim)/2, 
                          name="penultimate_layer"))
      
      # Then apply further linear layers to output latent mu and log sigma
      mu = tf.layers.dense(hidden, self.latent_dim, name="mean_layer")
      log_sigma = tf.layers.dense(hidden, self.latent_dim, name="std_layer")
      

    # Compute sigma
    sigma = 0.1 + 0.9 * tf.sigmoid(log_sigma)

    return GaussianParams(mu, sigma)

In [0]:
params = NeuralProcessParams(dim_z=20, n_hidden_units_h=[128, 128, 128], n_hidden_units_g=[128, 128, 128])

class NLP_NeuralProcess(object):
  
  def __init__(self, 
                  params = NeuralProcessParams(dim_z=20, 
                                                     n_hidden_units_h=[128, 128, 128], 
                                                     n_hidden_units_g=[128, 128, 128]),
                  num_classes = 6, 
                  num_draws = 2
                 ):
    
    self.params = params
    self.encoder = Encoder(layer_dims = self.params.n_hidden_units_h, 
                           latent_dim=self.params.dim_z)
    self.decoder = Decoder(layer_dims= self.params.n_hidden_units_g, 
                           num_classes=num_classes)
    self.num_draws = num_draws
    #self.embedder = Embedder()
    
  def create_model(self, input_ids, input_mask, segment_ids, scores=None):
    
    # apply embedder
    embedder = hub.Module(BERT_model_hub,trainable=True)
    bert_inputs = dict(input_ids=input_ids,
                       input_mask=input_mask, 
                       segment_ids=segment_ids)

    embeddings = embedder(inputs=bert_inputs,
                               signature="tokens", 
                               as_dict=True)

    tf.logging.info(embeddings)
    
    # total x,y 
    x_all = embeddings["pooled_output"]
    y_all = scores
    
    # context split
    context_xs, context_ys, target_xs, target_ys = self.context_target_split(x_all, y_all, batch_size =32)
    
    # get encoding params
    context_z_dist = self.encoder(context_xs, context_ys)
    all_z_dist = self.encoder(x_all, y_all)
    
    # get internal representation
    mean_zero = tf.constant(np.repeat(0., params.dim_z))
    epsilon_dist = tfd.MultivariateNormalDiag(loc= mean_zero)                            
    epsilon = tf.expand_dims(epsilon_dist.sample(self.num_draws), axis=1)
    epsilon = tf.cast(epsilon, tf.float32)
    # context representation with target xs
    y_pred = self.decoder(target_xs, context_z_dist.sample(self.num_draws))
    prior_predict = self.decoder(x_all, epsilon)
   
    # loss
    loglike = self.loglikelihood(target_ys, y_pred)
    KL_loss = self.KLqp_gaussian(all_z_dist.parameters['loc'], 
                                 all_z_dist.parameters['scale_diag'], 
                                 context_z_dist.parameters['loc'], 
                                 context_z_dist.parameters['scale_diag'])
    loss = tf.negative(loglike) + KL_loss
    
    return (loss, y_pred, prior_predict, y_all)
  
  def context_target_split(self, xs, ys, batch_size =32):
    btch_sz = tf.minimum(tf.constant(batch_size), tf.shape(xs)[0])
    n_context = tf.random_shuffle(tf.range(1,btch_sz))[0]
    
    indices = tf.range(0, btch_sz)
    context_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context))
    target_set_indices = tf.gather(tf.random_shuffle(indices),tf.range(n_context, btch_sz))
    
    context_xs = tf.gather(xs,context_set_indices)
    context_ys = tf.gather(ys, context_set_indices)
    target_xs = tf.gather(xs,target_set_indices)
    target_ys = tf.gather(ys,target_set_indices)
    
    return context_xs, context_ys, target_xs, target_ys
  
  def loglikelihood(self, y_star: tf.Tensor, dist):
    """Log-likelihood of an output given a predicted """
    p_normal = tfd.MultivariateNormalDiag(loc = dist.mu, scale_diag=dist.sigma)
    loglike = p_normal.log_prob(y_star)
    loglike = tf.reduce_sum(loglike, axis=0)
    loglike = tf.reduce_mean(loglike)
    return loglike
  
  def KLqp_gaussian(self, mu_q: tf.Tensor, sigma_q: tf.Tensor, mu_p: tf.Tensor, sigma_p: tf.Tensor) -> tf.Tensor:
    """Kullback-Leibler divergence between two Gaussian distributions

    Determines KL(q || p) = < log( q / p ) >_q

    Parameters
    ----------
    mu_q
        Mean tensor of distribution q, shape: (1, dim)
    sigma_q
        Variance tensor of distribution q, shape: (1, dim)
    mu_p
        Mean tensor of distribution p, shape: (1, dim)
    sigma_p
        Variance tensor of distribution p, shape: (1, dim)

    Returns
    -------
        KL tensor, shape: (1)
    """
    sigma2_q = tf.square(sigma_q) + 1e-16
    sigma2_p = tf.square(sigma_p) + 1e-16
    temp = sigma2_q / sigma2_p + tf.square(mu_q - mu_p) / sigma2_p - 1.0 + tf.log(sigma2_p / sigma2_q + 1e-16)
    return 0.5 * tf.reduce_sum(temp)
  
  
  def model_fn_builder(self, num_labels, learning_rate, num_train_steps, num_warmup_steps):
      """Returns `model_fn` closure for TPUEstimator."""
      def model_fn(features, mode, params):  # pylint: disable=unused-argument
          """The `model_fn` for TPUEstimator."""

          input_ids = features["input_ids"]
          input_mask = features["input_mask"]
          segment_ids = features["segment_ids"]    
          scores = features["scores"]



          (loss, posterior_predict, prior_predict, true_y) = self.create_model(input_ids, input_mask, segment_ids, scores)


          train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

          # output from model
          ystar, variance = tf.nn.moments(posterior_predict.mu,[0])
          prior_ystar, prior_variance = tf.nn.moments(prior_predict.mu,[0])
          print(prior_ystar)
          print(true_y)
          
          # Calculate evaluation metrics.
          eval_metrics = {}

          # AUC
          def metric_fn(pred_scores, real_scores, trait_num):
              auc_value = tf.metrics.auc(real_scores[:,trait_num], pred_scores[:,trait_num])
              accuracy_value = tf.metrics.accuracy(labels = tf.round(real_scores[:,trait_num]), predictions=tf.round(pred_scores[:,trait_num]))
              return {"auc"+str(trait_num): auc_value, "accuracy"+str(trait_num): accuracy_value}

          labels = true_y # need to round them if true labels are not 1 or 0
          eval_metrics_lst = [metric_fn(prior_ystar, labels, trait_num) for trait_num in range(num_labels)]

          for d in eval_metrics_lst:
              tf.summary.scalar(list(d.keys())[0], list(d.values())[0][1]) # make available to tensorboard
              eval_metrics.update(d)


          # returns 
          if mode == tf.estimator.ModeKeys.TRAIN:
              return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)


          elif mode == tf.estimator.ModeKeys.PREDICT:
              return tf.estimator.EstimatorSpec(mode=mode, predictions={'prediction_mean': prior_ystar, 'prediction_var': prior_variance})


          else:
              return tf.estimator.EstimatorSpec(mode=mode, eval_metric_ops=eval_metrics)

      return model_fn

In [0]:

tf.reset_default_graph()
neural_process = NLP_NeuralProcess()

In [0]:
dist = tfd.MultivariateNormalDiag(loc = np.repeat(0., 10))
sess = tf.Session()
param = GaussianParams(mu=np.array([1.,2.,3.]), sigma=np.array([1.,1.,1.]))

tf.expand_dims(dist.sample(2), axis=1)

dist.parameters['scale_diag']

In [0]:
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
batch_size = 32
# lr = 2e-2
lr = 2e-5
epochs = 6.0
# Warmup is a period of time where the learning rate  is small and gradually increases
warmpup_proportion = 0.1

# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / batch_size *epochs)
num_warmup_steps = int(num_train_steps * warmpup_proportion)

#####

output_dir = "./test_output"
save_checkpoints_steps = 500
save_summary_steps = 100

# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(model_dir=output_dir,
    save_summary_steps=save_summary_steps, save_checkpoints_steps=save_checkpoints_steps)

#####

num_labels = 6

model_fn = neural_process.model_fn_builder(num_labels = num_labels, learning_rate=lr,
  num_train_steps=num_train_steps, num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(model_fn=model_fn,config=run_config,
  params={"batch_size": batch_size})

#####

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = input_fn_builder(
    features=train_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=True, drop_remainder=False)

#####

print('Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

INFO:tensorflow:Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fee808d9160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0525 11:10:00.554350 140665273620352 estimator.py:201] Using config: {'_model_dir': './test_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fee808d9160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Beginning Training!
INFO:tensorflow:Calling model_fn.


I0525 11:10:10.616548 140665273620352 estimator.py:1111] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

I0525 11:25:41.816960 140665273620352 basic_session_run_hooks.py:594] Saving checkpoints for 1000 into ./test_output/model.ckpt.


INFO:tensorflow:global_step/sec: 1.05213


I0525 11:25:52.374772 140665273620352 basic_session_run_hooks.py:680] global_step/sec: 1.05213


INFO:tensorflow:loss = -1.5108039, step = 1000 (95.043 sec)


I0525 11:25:52.377210 140665273620352 basic_session_run_hooks.py:247] loss = -1.5108039, step = 1000 (95.043 sec)


In [0]:
! cp -r ./test_output "./gdrive/My Drive/Data/"

In [0]:
eval_steps = int(len(df_test) / batch_size)
eval_input_fn = input_fn_builder(
    features=test_features,
    seq_length=max_seq_length,
    num_labels = num_labels,
    is_training=False,
    drop_remainder=True)
result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

In [0]:
print("***** Eval results *****")
for key in sorted(result.keys()):
  print('  {} = {}'.format(key, str(result[key])))
      

In [0]:
predict_features = test_features[:100]

In [0]:
test_input_fn = input_fn_builder(
    features=predict_features, seq_length=max_seq_length, 
    num_labels = num_labels, is_training=False, drop_remainder=False)

In [0]:
test_scores = np.array([predict_features[i].score for i, val in enumerate(predict_features)])

preds = estimator.predict(input_fn=test_input_fn)
preds_vals = list(preds)
pred_means = np.array([val['prediction_mean'] for key, val in enumerate(preds_vals)])

In [0]:
def total_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = np.reshape(preds == test_scores,-1)
  return np.sum(correct_guesses) / len(correct_guesses)

In [0]:
def class_accuaracy(pred_means, test_scores):
  preds = np.round(pred_means)
  correct_guesses = preds == test_scores
  return correct_guesses.sum(axis=0)/ len(correct_guesses)


In [0]:
print(total_accuaracy(pred_means, test_scores))
print(class_accuaracy(pred_means, test_scores))

In [0]:
preds_vals

In [0]:
test_scores